In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv,to_hetero
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv
from torch_geometric.loader import NeighborLoader
from torch_geometric.data import Data

In [7]:
import pandas as pd
import torch
import torch_geometric
from torch_geometric.data import HeteroData
from torch_geometric.nn import SAGEConv, to_hetero
from torch_geometric.loader import HGTLoader
import torch.nn.functional as F
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Load the data
movies = pd.read_csv('/home/aakash/Documents/COding/Recommender/ml-20m/movies.csv')
ratings = pd.read_csv('/home/aakash/Documents/COding/Recommender/ml-20m/ratings.csv')
links = pd.read_csv('/home/aakash/Documents/COding/Recommender/ml-20m/links.csv')
tags = pd.read_csv('/home/aakash/Documents/COding/Recommender/ml-20m/tags.csv')
genome_scores = pd.read_csv('/home/aakash/Documents/COding/Recommender/ml-20m/genome-scores.csv')
genome_tags = pd.read_csv('/home/aakash/Documents/COding/Recommender/ml-20m/genome-tags.csv')
user_encoder = LabelEncoder()
movie_encoder = LabelEncoder()

ratings['userId'] = user_encoder.fit_transform(ratings['userId'])
ratings['movieId'] = movie_encoder.fit_transform(ratings['movieId'])

# Create edge index
edge_index = torch.tensor(ratings[['userId', 'movieId']].values.T, dtype=torch.long)

# Node features (optional): for simplicity, we use identity matrix as features
num_users = ratings['userId'].nunique()
num_movies = ratings['movieId'].nunique()
num_nodes = num_users + num_movies
# Set embedding dimensions
embedding_dim = 32  # Example dimension

# Create embeddings instead of one-hot encoding
user_embeddings = nn.Embedding(num_embeddings=num_users, embedding_dim=embedding_dim)
movie_embeddings = nn.Embedding(num_embeddings=num_movies, embedding_dim=embedding_dim)


# Initialize embeddings
user_features = user_embeddings(torch.arange(num_users))
movie_features = movie_embeddings(torch.arange(num_movies))


x = torch.cat([user_features, movie_features], dim=0)

# Ratings as edge labels
edge_labels = torch.tensor(ratings['rating'].values, dtype=torch.float)

# Create the graph data object
data = Data(x=x, edge_index=edge_index, edge_attr=edge_labels)

In [8]:
print(data)

Data(x=[165237, 32], edge_index=[2, 20000263], edge_attr=[20000263])


In [9]:
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv
from torch_geometric.loader import NeighborLoader

class GraphSAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

# Initialize the model, optimizer, and loss function
model = GraphSAGE(in_channels=x.size(1), hidden_channels=32, out_channels=16)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()


In [10]:
def train():
    model.train()

    total_loss = 0
    for data in loader:  # loader is a NeighborLoader
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        out = out[data.edge_index[0]] * out[data.edge_index[1]]  # Dot product of user and movie embeddings
        loss = criterion(out.sum(dim=1), data.edge_attr)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(loader)

# Using NeighborLoader for mini-batch training
loader = NeighborLoader(data, num_neighbors=[10, 10], batch_size=1024, shuffle=True)

# Training the model
for epoch in range(1, 101):
    loss = train()
    print(f'Epoch {epoch:03d}, Loss: {loss:.4f}')


/home/aakash/miniconda3/envs/test/lib/python3.9/site-packages/torch_geometric/sampler/neighbor_sampler.py:61: UserWarning: Using 'NeighborSampler' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  warnings.warn(f"Using '{self.__class__.__name__}' without a "


RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.